In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [5]:
PREFER_CUDA = True

In [6]:
torch.cuda.is_available()

True

In [17]:
use_cuda = PREFER_CUDA and torch.cuda.is_available()

# args.seed
batch_size = 100
nb_epochs = 5

'''kwargs = {'num_workers': 1, 'pin_memory': True} if else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   
    batch_size=, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       
    batch_size=, shuffle=True, **kwargs)
'''

vocab_size = 250
n_words = 3
n_embed = 50
n_hidden = 200
n_output = vocab_size


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.E2H = nn.Linear(n_words*n_embed, n_hidden, bias=True)
        self.H2O = nn.Linear(n_hidden, n_output, bias=True)
        #self.conv2_drop = nn.Dropout2d()

    def forward(self, x):
        h_activation = self.E2H(x)
        h = F.sigmoid(h_activation)
        out_activation = self.E2H(h)
        return F.softmax(out_activation)
        #x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        #x = x.view(-1, 320)
        #x = F.relu(self.fc1(x))
        #x = F.dropout(x, training=self.training)
        #x = F.relu(self.fc2(x))
        #return F.log_softmax(x)

model = Net()
if use_cuda:
    model.cuda()

#optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        #loss = F.nll_loss(output, target)
        loss = F.cross_entropy(output, target, size_average=True)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss = test_loss
    test_loss /= len(test_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, nb_epochs + 1):
    #train(epoch)
    #test(epoch)
    None
